# OpenStreetMap Nominatim API

## Nominatim API の概要

- Nominatim API とは、 OpenStreetMap のデータを利用して、地名から緯度経度を取得することができる API です
  - https://nominatim.org/
  - https://nominatim.openstreetmap.org/ui/search.html
- 他にも様々な機能がありますが、詳細は公式ドキュメントを参照してください
  - https://nominatim.org/release-docs/latest/
  - https://nominatim.org/release-docs/latest/api/Overview/
  - https://nominatim.org/release-docs/latest/api/Search/

## 注意

- このノートブックでは、 nominatim.openstreetmap.org で運営されている Nominatim API を呼び出しています
  - 以下の利用ポリシーを必ず読んで、従ってください
    - https://operations.osmfoundation.org/policies/nominatim/
- nominatim.openstreetmap.org で運営されている Nominatim API は、 **一秒間に最大で一つのリクエスト** に制限されています
  - この頻度を超えるリクエストが必要な場合は、自分で Nominatim API サーバーを立てることができます
  - インストール方法は公式ドキュメントを参照してください
    - https://nominatim.org/release-docs/latest/admin/Installation/


## このノートブックの流れ

- 生成AIに、自然言語から地名のみを出力させる
- 出力された地名に基づいて Nominatim API を呼び出して GPS 座標を取得する
- 取得した GPS 座標を地図上に表示する

## 前準備

### 環境変数の読み込み

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### 出力を大きく表示する関数の用意

In [2]:
from IPython.display import Markdown, display
def print_large(text):
    display(Markdown(f"### {text}"))

## 対象となる自然言語

In [3]:
input_text = "出島メッセ長崎に行きたいです。"
print_large(input_text)

### 出島メッセ長崎に行きたいです。

## 生成 AI に自然言語から地名のみを出力させる

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# モデルの準備
model = ChatGoogleGenerativeAI(model="gemini-exp-1206", temperature=0)

# プロンプトの準備
template = """Extract only the first place name from the user input.
Output only the place name.

User input:
{input}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

res = chain.invoke({"input": input_text})
result = res.content.strip()
print_large(result)

### 出島メッセ長崎

## 出力された地名に基づいて Nominatim API を呼び出して GPS 座標を取得

In [5]:
import requests

def get_nominatim_response(query: str):
    # URLパラメータを設定
    params = {
        "format": "jsonv2",
        "polygon_geojson": "0",
        "q": query
    }
    # ヘッダー（特にUser-Agent）を設定
    headers = {
        "User-Agent": "NLP2025/1.0"
    }
    # APIリクエストを送信
    nominatim_url = "https://nominatim.openstreetmap.org/search"
    response = requests.get(nominatim_url, params=params, headers=headers)
    if response.status_code == 200:
        res_json = response.json()
        return res_json
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

In [6]:
import json
nominatim_response = get_nominatim_response(result)
print(json.dumps(nominatim_response, indent=2, ensure_ascii=False))

lat = nominatim_response[0]["lat"]
lon = nominatim_response[0]["lon"]
bbox = nominatim_response[0]["boundingbox"]
print_large(f"Latitude: {lat}, Longitude: {lon}")

[
  {
    "place_id": 249452341,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
    "osm_type": "way",
    "osm_id": 999078939,
    "lat": "32.752506600000004",
    "lon": "129.86764840717382",
    "category": "amenity",
    "type": "exhibition_centre",
    "place_rank": 30,
    "importance": 6.96322575722441e-05,
    "addresstype": "amenity",
    "name": "出島メッセ長崎",
    "display_name": "出島メッセ長崎, 長崎式見港線, 尾上町, 長崎市, 長崎県, 850-8685, 日本",
    "boundingbox": [
      "32.7515478",
      "32.7533039",
      "129.8672832",
      "129.8681479"
    ]
  }
]


### Latitude: 32.752506600000004, Longitude: 129.86764840717382

## Nominatim API で取得したデータを GeoJSON に変換する

In [7]:
geojson_data = {
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [float(lon), float(lat)]
    },
    "properties": {
        "title": result
    }
}

## GeoJSON を地図上に表示する

In [8]:
import folium

# default map
m = folium.Map(location=[0, 0], zoom_start=2)

# Add the GeoJSON data to the map
folium.GeoJson(geojson_data).add_to(m)

# bbox で fit_bounds
if bbox:
    south, north, west, east = map(float, bbox)
    m.fit_bounds([[south, west], [north, east]])

# 地図を表示
display(m)

## まとめ

- 生成 AI は、自然言語から地名のみを出力することができる
- 出力された地名に基づいて Nominatim API を呼び出して GPS 座標を取得することができる
- 取得した GPS 座標を地図上にプロットすることができる